In [1]:
import os
import time
import math
from datetime import datetime as dt
from trading_helper import TradingHelper
if os.path.exists("operar_MR36O_LECHO.log"):os.remove("operar_MR36O_LECHO.log")
app = TradingHelper(log_name="operar_MR36O_LECHO")
mail = "cirigliano.santiago@gmail.com"
cantidad_minima = 5
terms = ["24hs", "CI"]
maximum_vol = 100
lista = ["MR36O", "LECHO"]
combinations = (lista, lista[::-1])  # Original y lista invertida
while dt.now().hour < 11:
    time.sleep(60)
app.logger.info(f"arrancando {dt.now()}")
while 11 <= dt.now().hour < 17:
    try:
        if dt.now().hour >= 17:
            break
        for position in [0, 1]:
            for term in terms:
                tickers = combinations[position]
                time.sleep(0.5)
                try:
                    amount_in_portfolio = app.stocks_available(mail=mail, ticker=tickers[0], term=term, currency="pesos")[term]
                except Exception as e:
                    app.logger.error(f"Error al obtener stocks disponibles para {tickers[0]}: {e}")
                    amount_in_portfolio = app.stocks_available(mail=mail, ticker=tickers[0], term=term, currency="pesos")[term]

                if amount_in_portfolio != 0:
                    try:
                        numerador = app.get_snapshot(mail=mail, ticker=tickers[0])
                        denominador = app.get_snapshot(mail=mail, ticker=tickers[1])
                        while not isinstance(numerador, list) or not isinstance(denominador, list):
                            numerador = app.get_snapshot(mail=mail, ticker=tickers[0])
                            denominador = app.get_snapshot(mail=mail, ticker=tickers[1])
                            time.sleep(0.5)
                    except Exception as e:
                        app.logger.error(f"Error al obtener snapshot para {tickers}: {e}")
                        while not isinstance(numerador, list) or not isinstance(denominador, list):
                            try:
                                numerador = app.get_snapshot(mail=mail, ticker=tickers[0])
                                denominador = app.get_snapshot(mail=mail, ticker=tickers[1])
                                break
                            except Exception as e:
                                if "Error code: 500" not in str(e):
                                    app.logger.error(f"Error no 500 al obtener snapshot: {e}")
                                    break
                                continue

                    numerador = [item for item in numerador if item['currency'] == 'ARS' and item['term'] == term][0]
                    denominador = [item for item in denominador if item['currency'] == 'ARS' and item['term'] == term][0]
                    
                    precio_venta_numerador = numerador['bids'][0]['price']
                    vol_venta_numerador = numerador['bids'][0]['size']
                    precio_compra_denominador = denominador['asks'][0]['price']
                    vol_compra_denominador = denominador['asks'][0]['size']
                    
                    app.logger.info(f"{tickers[0]}, {term}, {amount_in_portfolio}")
                    amount_in_portfolio = min(amount_in_portfolio, maximum_vol)
                    minimum_vol = min(vol_compra_denominador, vol_venta_numerador, amount_in_portfolio)

                    if precio_venta_numerador == 0 or precio_compra_denominador == 0 or minimum_vol == 0:
                        break

                    ratio = precio_venta_numerador / precio_compra_denominador
                    if ratio >= 1.005:
                        maximum_vol = 201
                    if tickers[0]=="LECHO": min_ratio=0.99999
                    elif tickers[0]=="MR36O": min_ratio = 1.003

                    app.logger.info(f"Ratio {ratio}, vendiendo {vol_venta_numerador} de {tickers[0]} a {precio_venta_numerador} y comprando {vol_compra_denominador} de {tickers[1]} a {precio_compra_denominador}")

                    if ratio > min_ratio:
                        if minimum_vol == vol_venta_numerador:
                            vol_compra_denominador = math.floor(vol_venta_numerador * ratio)
                        elif minimum_vol == amount_in_portfolio:
                            vol_venta_numerador = amount_in_portfolio
                            vol_compra_denominador = math.floor(vol_venta_numerador * ratio)
                        elif minimum_vol == vol_compra_denominador:
                            vol_venta_numerador = math.ceil(vol_compra_denominador / ratio)

                        if vol_venta_numerador * precio_venta_numerador < vol_compra_denominador * precio_compra_denominador:
                            vol_compra_denominador -= 1

                        app.logger.info(f"Oportunidad ratio {ratio}, vendiendo {vol_venta_numerador} de {tickers[0]} a {precio_venta_numerador} y comprando {vol_compra_denominador} de {tickers[1]} a {precio_compra_denominador}")

                        sell_order = app.place_sell_order(mail=mail,
                                                          ticker=tickers[0],
                                                          quantity=vol_venta_numerador,
                                                          price=precio_venta_numerador,
                                                          term=term,
                                                          currency="pesos")
                        app.logger.info(f"Orden de venta {tickers[0]}: {sell_order}")

                        order_status = app.check_order_status(mail=mail, order_number=sell_order["Orden"])
                        app.logger.info(f"Estado de la orden de venta: {order_status}")

                        start_time = time.time()
                        while order_status == "MARKET":
                            elapsed_time = time.time() - start_time
                            if elapsed_time > 4:
                                cancel_order = app.cancel_order(mail=mail, order_number=sell_order["Orden"])
                                app.logger.info(f"Orden cancelada por Timeout: {cancel_order}")
                                break
                            order_status = app.check_order_status(mail=mail, order_number=sell_order["Orden"])
                            app.logger.info(f"Estado de la orden de venta: {order_status}")

                        if order_status == "EXECUTED":
                            buy_order = app.place_buy_order(mail=mail,
                                                            ticker=tickers[1],
                                                            quantity=vol_compra_denominador,
                                                            price=precio_compra_denominador,
                                                            term=term,
                                                            currency="pesos")
                            app.logger.info(f"Orden de compra {tickers[1]}: {buy_order}")

                            order_status = app.check_order_status(mail=mail, order_number=buy_order["Orden"])
                            while order_status == "MARKET":
                                try:
                                    time.sleep(1)
                                    order_status = app.check_order_status(mail=mail, order_number=buy_order["Orden"])
                                    if order_status == "FILLED":
                                        break
                                    cancel_order = app.cancel_order(mail=mail, order_number=buy_order["Orden"])
                                    app.logger.info(f"Orden de compra cancelada: {cancel_order}")
                                    precio_compra_denominador -= 0.1
                                    buy_order = app.place_buy_order(mail=mail,
                                                                    ticker=tickers[1],
                                                                    quantity=vol_compra_denominador,
                                                                    price=precio_compra_denominador,
                                                                    term=term,
                                                                    currency="pesos")
                                except Exception as e:
                                    app.logger.error(f"Error en la compra de {tickers[1]}: {e}")

                            app.logger.info(f"Estado de la orden de compra: {order_status}")
                            if buy_order["Success"]:
                                app.logger.info(f"Negocio exitoso con {tickers[1]}")

    except Exception as e:
        app.logger.error(f"Error general: {e}")

401 error: Refreshing access token
401 error: Refreshing access token
401 error: Refreshing access token
401 error: Refreshing access token


In [ ]:
#Gestionar la siguiente situacion de venta
"""Ratio 1.0051187260059717 vendiendo 130 de MR36O a precio venta 70690 comprando 266 de LECHO a precio compra 70330
Oportunidad ratio 1.0051187260059717 vendiendo 130 de MR36O a precio venta 70690 y comprando 130 de LECHO a precio venta 70330
Orden de venta MR36O {'Success': True, 'Orden': 'e2c64284-d552-4ac9-86b8-5adfc48f571e'}
PARTIALLY_EXECUTED
LECHO, 24hs, 885"""

"Ratio 1.0051187260059717 vendiendo 130 de MR36O a precio venta 70690 comprando 266 de LECHO a precio compra 70330\nOportunidad ratio 1.0051187260059717 vendiendo 130 de MR36O a precio venta 70690 y comprando 130 de LECHO a precio venta 70330\nOrden de venta MR36O {'Success': True, 'Orden': 'e2c64284-d552-4ac9-86b8-5adfc48f571e'}\nPARTIALLY_EXECUTED\nLECHO, 24hs, 885"